In [2]:
import numpy as np

import pandas as pd

from tqdm import tqdm

from fuzzywuzzy import fuzz
from itertools import product

In [3]:
datapath = storepath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\04_DATA\\STF.xlsx'

In [43]:
bf = pd.read_excel(datapath,index_col=0,parse_dates=['Date_booked','Date_ordered'])

In [44]:
df = bf.drop(columns=['Use'])

for colname in df.columns: 
    if df[colname].dtype=='object':
        df[colname] = df[colname].str.lower()
        
df['DUPLICATE'] = -1

In [45]:
SIM_THRESHOLD = 0

In [46]:
daterange = list(pd.date_range(df['Date_booked'].min(),df['Date_booked'].max(),freq='D'))

d_flag = 1
for c_date in tqdm(daterange):

    tf = df[(df['Date_booked']==c_date)].drop(columns=['Date_booked'])

    for colname in tf.columns:
        if len(tf[colname].unique())==1: tf = tf.drop(columns=[colname])
    I = []
    for k in list(product(tf.index.tolist(),tf.index.tolist())):
        if k[0]!=k[1] and (k[1],k[0]) not in I: I.append(list(k))

    for ii in I:
        if df.loc[ii,'DUPLICATE'].sum()!=-2: continue
        sim = []
        for X in tf.loc[ii].transpose().values:
            if pd.isnull(X[0]): sim.append(abs(1 - pd.isnull(X[0])))
            elif pd.isnull(X[1]): sim.append(abs(1 - pd.isnull(X[1])))
            elif type(X[0]) is str: sim.append(abs(fuzz.ratio(X[0],X[1]))/100)
            elif type(X[0]) is type(pd.to_datetime('2020-01-01')): sim.append(abs((X[0]-X[1]).days))
            else: sim.append(abs(X[0]-X[1])/10)
        if sum(sim)<=SIM_THRESHOLD: 
            df.loc[ii,'DUPLICATE'] = d_flag
            d_flag = d_flag + 1
bf['DUPLICATE'] = df['DUPLICATE']

100%|████████████████████████████████████████| 511/511 [00:23<00:00, 21.50it/s]


In [47]:
c_id = 7

str0 = bf[bf['DUPLICATE']==c_id].iloc[0]['Use'].lstrip().rstrip()
str1 = bf[bf['DUPLICATE']==c_id].iloc[1]['Use'].lstrip().rstrip()

In [48]:
str0

'PP 8381 PP SPOTIFY Ihr Einkauf bei SPOTIFY'

In [49]:
str1

'EREF 1005350549298 PP 8381 PP PAYPALMREF 4CP2224PF86NECRED LU96ZZZ0000000000000000058SVWZ PP 8381 PP SPOTIFY Ihr Einkauf bei SPOTIFY'

In [68]:
bf[(bf['DUPLICATE']>0)&~(bf['Use'].apply(lambda x: 'prime' in x.lower() or 'amzn mktp' in x.lower()))].index

Int64Index([ 15,  28,  77,  86,  87, 118, 146, 147, 155, 194, 221, 229, 232,
            233, 250, 251, 252, 256, 352, 360, 374, 379, 389, 394, 420, 421,
            422, 430, 472, 499, 533, 562, 575, 620, 629, 630, 659, 686, 687,
            695, 710, 718, 721, 722, 737, 738, 739, 743, 771, 779, 792, 797,
            807, 812, 836, 837, 838, 846, 886, 911],
           dtype='int64')

In [67]:
bf[bf['DUPLICATE']>0]

,OrderAccount,Date_booked,Date_ordered,Text_transaction,Use,Contact,AccNum,BIC,Value_transaction,Currency,Information,SpendingType,ExpenseGroup,DUPLICATE
10,DE98773501100020557849,2019-09-26,2019-09-26,FOLGELASTSCHRIFT,D01 3386999 4642225 Prime Video 7EIIAB22V8IHBJFC,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-3.99,EUR,Umsatz gebucht,AMAZONVIDEOS,AMAZON,36
15,DE98773501100020557849,2019-09-23,2019-09-23,EIGENE KREDITKARTENABRECHN,VISA ABR 490762XXXXXX5575,KREDITKARTENABRECHNUNG,9902292060,77350110,-245.86,EUR,Umsatz gebucht,CREDITCARD,CREDITCARDBILL,35
28,DE98773501100020557849,2019-09-04,2019-09-04,FOLGELASTSCHRIFT,Kieser Training Beitrag ID238912 VOM 03 09 2019,JENS MOELLER GMBH,DE85763500000000001669,BYLADEM1ERH,-69.00,EUR,Umsatz gebucht,MEMBERSHIP_FEES KIESER,MEMBERSHIP_FEES,34
40,DE98773501100020557849,2019-08-27,2019-08-27,FOLGELASTSCHRIFT,D01 6278352 4568626 Prime Video 1R29MPD0NUAZ417C,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-5.98,EUR,Umsatz gebucht,AMAZONVIDEOS,AMAZON,33
48,DE98773501100020557849,2019-08-20,2019-08-20,FOLGELASTSCHRIFT,D01 8161227 1743057 Prime Video 74Q50QIR3FUSSRCL,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-3.99,EUR,Umsatz gebucht,AMAZONVIDEOS,AMAZON,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,DE98773501100020557849,2019-12-13,2019-12-13,FOLGELASTSCHRIFT,302 7599093 1798758 AMZN Mktp DE 53289PG8875QZ...,AMAZON PAYMENTS EUROPE S C A,DE87300308801908262006,TUBDDEDD,-25.99,EUR,Umsatz gebucht,AMAZONSHOPPING,AMAZON,42
875,DE98773501100020557849,2019-12-13,2019-12-13,FOLGELASTSCHRIFT,302 6220836 5954721 AMZN Mktp DE 4GMSKXARK6IF1...,AMAZON PAYMENTS EUROPE S C A,DE87300308801908262006,TUBDDEDD,-2.71,EUR,Umsatz gebucht,AMAZONSHOPPING,AMAZON,43
876,DE98773501100020557849,2019-12-13,2019-12-13,FOLGELASTSCHRIFT,302 4702496 6358767 AMZN Mktp DE 2L65ZNHIIRLJX...,AMAZON PAYMENTS EUROPE S C A,DE87300308801908262006,TUBDDEDD,-7.99,EUR,Umsatz gebucht,AMAZONSHOPPING,AMAZON,44
886,DE98773501100020557849,2019-12-04,2019-12-04,FOLGELASTSCHRIFT,Kieser Training Beitrag ID238915 VOM 03 12 2019,JENS MOELLER GMBH,DE85763500000000001669,BYLADEM1ERH,-69.00,EUR,Umsatz gebucht,MEMBERSHIP_FEES KIESER,MEMBERSHIP_FEES,41
